In [1]:
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.realestate.com.au/international/us/rent'

browser = webdriver.Chrome()
browser.get(url)
browser.implicitly_wait(20)

In [3]:
houses = pd.DataFrame(columns=[
    'Price', 
    'Rooms',
    'Type',
    'Status',
    'Building Size',
    'Architecture Style', 
    'County',
    'Provider',
])

In [4]:
while len(houses) < 1000:
    links = browser.find_elements(By.XPATH, '//div[@class="sc-1dun5hk-0 cOiOrj"]/a')
    urls = [link.get_attribute('href') for link in links]
    
    for sub_url in urls:
        browser.execute_script("window.open(arguments[0], '_blank');", sub_url)
        browser.switch_to.window(browser.window_handles[1]) 
        
        try:
            price = browser.find_element(By.XPATH, '//div[@class="sc-10v3xoh-1 cqrlhJ"]').text
            info_list = browser.find_elements(By.XPATH, '//div[contains(@class, "sc-12iqlu8-0")]')
            
            basic_info = {}
            for info in info_list:
                key = info.find_element(By.XPATH, './/div[@class="sc-12iqlu8-1 Mguuo basicInfoKey"]').text
                value = info.find_element(By.XPATH, './/div[@class="sc-12iqlu8-2 dPeBQf basicInfoValue"]').text
                basic_info[key] = value
            
            house = pd.Series([
                price,
                basic_info.get('Rooms', None),
                basic_info.get('Property Type', None),
                basic_info.get('Listing Status', None),
                basic_info.get('Building Size', None),
                basic_info.get('Architecture Style', None),
                basic_info.get('County', None),
                basic_info.get('Provider', None)
            ], index=houses.columns)
            
            houses.loc[len(houses)] = house
        except Exception as e:
            print(f"Error occurred while scraping {sub_url}: {e}")
        
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        
        time.sleep(1)
        print(sub_url)

    next_page = browser.find_element(By.XPATH, '//li[@title="Next Page"]/a')
    next_url = next_page.get_attribute('href')
    browser.get(next_url)

houses

https://www.realestate.com.au/international/us/1469-terrace-drive-bullhead-city-az-110095246651/
https://www.realestate.com.au/international/us/164-richmond-street-apt-2a-philadelphia-pa-110095236292/
https://www.realestate.com.au/international/us/1155-s-15th-street-apt-unit-401-philadelphia-pa-110095236291/
https://www.realestate.com.au/international/us/2551-bonmark-drive-ojai-ca-110095236267/
https://www.realestate.com.au/international/us/8609-coral-gables-lane-vienna-va-110095236257/
https://www.realestate.com.au/international/us/902-8th-street-laurel-md-110095236213/
https://www.realestate.com.au/international/us/720-fayette-street-apt-2nd-floor-front-conshohocken-pa-110095236214/
https://www.realestate.com.au/international/us/222-church-street-apt-2a-philadelphia-pa-110095236200/
https://www.realestate.com.au/international/us/866-hendrix-street-philadelphia-pa-110095236192/
https://www.realestate.com.au/international/us/1701-tulip-street-apt-102-philadelphia-pa-110095236196/
https

KeyboardInterrupt: 

In [5]:
houses

,Price,Rooms,Type,Status,Building Size,Architecture Style,County,Provider
0,"USD $2,200 per month","3 bedroom(s), 2 bathroom(s)",House,Active,136.20 m2,Other,Mohave,"Black Mountain Valley Realty, Inc"
1,"USD $3,000 per month","2 bedroom(s), 3 bathroom(s)",Apartment,Active,205.13 m2,Other,PHILADELPHIA,Compass
2,"USD $1,195 per month","1 bedroom(s), 1 bathroom(s)",Apartment,Active,None,Other,PHILADELPHIA,OCF Realty
3,"USD $8,000 per month","4 bedroom(s), 3 bathroom(s)",Townhouse,Active,213.68 m2,Other,Ventura,eXp Realty of California Inc.
4,"USD $5,600 per month","4 bedroom(s), 3 bathroom(s), 1 basement(s)",House,Active,266.72 m2,Colonial,FAIRFAX,Keller Williams Fairfax Gateway
...,...,...,...,...,...,...,...,...
122,USD $900 per month,"1 bedroom(s), 1 bathroom(s)",Townhouse,Active,341.14 m2,Other,Fulton - GA,"Realtyka, LLC"
123,"USD $2,350 per month","3 bedroom(s), 3 bathroom(s)",House,Active,176.52 m2,New Traditional,Fulton - GA,Keller Williams Realty Cityside
124,"USD $1,895 per month","3 bedroom(s), 3 bathroom(s)",Other,Active,120.77 m2,Other,Fulton - GA,HomeSmart
125,"USD $2,650 per month","3 bedroom(s), 3 bathroom(s)",Other,Active,174.38 m2,Other,Lumpkin - GA,Virtual Properties Realty
